In [1]:
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options, ChromiumOptions

In [2]:
from pathlib import Path
import os

In [3]:
from util.repo import database_root, root, driver_root
db_test = os.path.join(database_root, "test.db")

In [14]:
# root = "/media/ywatcher/ExtDisk1/Files/py/archiv"
driver_root = os.path.join(root, "driver")
drivers = {
    "chrome" : os.path.join(
        # driver_root, 
        "chromedriver-linux64",
        # "/home/ywatcher/.cache/selenium/chromedriver/linux64/118.0.5993.70",
        "chromedriver"),
        
    "firefox" : os.path.join(
        driver_root, "geckodriver-v0.33.0-linux64", "geckodriver"),
    "edge": os.path.join(
        driver_root, "edgedriver_linux64", "msedgedriver")
}
# get drivers from https://googlechromelabs.github.io/chrome-for-testing/

In [15]:
t = "chrome"
# t = "firefox"
# t = "edge"
print(os.path.exists(drivers[t]))
print(Path(drivers[t]).is_file())

False
False


In [42]:
print(drivers[t])

/home/ywatcher/.cache/selenium/chromedriver/linux64/118.0.5993.70chromedriver


In [20]:
# from selenium.webdriver.common.driver_finder import DriverFinder
# from selenium.webdriver.common.service import Service
service = None  #Service()

_DriverFinder.get_path(service, options)

/media/ywatcher/ExtDisk1/Files/Packages/web_archiv/driver/chromedriver-linux64/chromedriver


'/media/ywatcher/ExtDisk1/Files/Packages/web_archiv/driver/chromedriver-linux64/chromedriver'

In [7]:
from selenium.webdriver.common.selenium_manager import SeleniumManager 

In [8]:
SeleniumManager().driver_location(options)

NameError: name 'options' is not defined

In [5]:
import typing

from selenium.types import SubprocessStdAlias
from selenium.webdriver.chromium import service


class Service(service.ChromiumService):
    """A Service class that is responsible for the starting and stopping of
    `chromedriver`.

    :param executable_path: install path of the chromedriver executable, defaults to `chromedriver`.
    :param port: Port for the service to run on, defaults to 0 where the operating system will decide.
    :param service_args: (Optional) List of args to be passed to the subprocess when launching the executable.
    :param log_output: (Optional) int representation of STDOUT/DEVNULL, any IO instance or String path to file.
    :param env: (Optional) Mapping of environment variables for the new process, defaults to `os.environ`.
    """

    def __init__(
        self,
        executable_path=None,
        port: int = 0,
        service_args: typing.Optional[typing.List[str]] = None,
        log_output: SubprocessStdAlias = None,
        env: typing.Optional[typing.Mapping[str, str]] = None,
        **kwargs,
    ) -> None:
        super().__init__(
            executable_path=executable_path,
            port=port,
            service_args=service_args,
            log_output=log_output,
            env=env,
            **kwargs,
        )
# Licensed to the Software Freedom Conservancy (SFC) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The SFC licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.

from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
# from selenium.webdriver.common.driver_finder import DriverFinder
from selenium.webdriver.common.options import ArgOptions

from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver


class ChromiumDriver(RemoteWebDriver):
    from selenium.webdriver.common.service import Service
    """Controls the WebDriver instance of ChromiumDriver and allows you to
    drive the browser."""

    def __init__(
        self,
        browser_name,
        vendor_prefix,
        options: ArgOptions,
        service: Service,
        keep_alive=True,
    ) -> None:
        """Creates a new WebDriver instance of the ChromiumDriver. Starts the
        service and then creates new WebDriver instance of ChromiumDriver.

        :Args:
         - browser_name - Browser name used when matching capabilities.
         - vendor_prefix - Company prefix to apply to vendor-specific WebDriver extension commands.
         - options - this takes an instance of ChromiumOptions
         - service - Service object for handling the browser driver if you need to pass extra details
         - keep_alive - Whether to configure ChromiumRemoteConnection to use HTTP keep-alive.
        """
        self.vendor_prefix = vendor_prefix

        self.service = service

        self.service.path = _DriverFinder.get_path(self.service, options)

        self.service.start()

        try:
            super().__init__(
                command_executor=ChromiumRemoteConnection(
                    remote_server_addr=self.service.service_url,
                    browser_name=browser_name,
                    vendor_prefix=vendor_prefix,
                    keep_alive=keep_alive,
                    ignore_proxy=options._ignore_local_proxy,
                ),
                options=options,
            )
        except Exception:
            self.quit()
            raise
        self._is_remote = False

    def launch_app(self, id):
        """Launches Chromium app specified by id."""
        return self.execute("launchApp", {"id": id})

    def get_network_conditions(self):
        """Gets Chromium network emulation settings.

        :Returns:     A dict. For example:     {'latency': 4,
        'download_throughput': 2, 'upload_throughput': 2,     'offline':
        False}
        """
        return self.execute("getNetworkConditions")["value"]

    def set_network_conditions(self, **network_conditions) -> None:
        """Sets Chromium network emulation settings.

        :Args:
         - network_conditions: A dict with conditions specification.

        :Usage:
            ::

                driver.set_network_conditions(
                    offline=False,
                    latency=5,  # additional latency (ms)
                    download_throughput=500 * 1024,  # maximal throughput
                    upload_throughput=500 * 1024)  # maximal throughput

            Note: 'throughput' can be used to set both (for download and upload).
        """
        self.execute("setNetworkConditions", {"network_conditions": network_conditions})

    def delete_network_conditions(self) -> None:
        """Resets Chromium network emulation settings."""
        self.execute("deleteNetworkConditions")

    def set_permissions(self, name: str, value: str) -> None:
        """Sets Applicable Permission.

        :Args:
         - name: The item to set the permission on.
         - value: The value to set on the item

        :Usage:
            ::

                driver.set_permissions('clipboard-read', 'denied')
        """
        self.execute("setPermissions", {"descriptor": {"name": name}, "state": value})

    def execute_cdp_cmd(self, cmd: str, cmd_args: dict):
        """Execute Chrome Devtools Protocol command and get returned result The
        command and command args should follow chrome devtools protocol
        domains/commands, refer to link
        https://chromedevtools.github.io/devtools-protocol/

        :Args:
         - cmd: A str, command name
         - cmd_args: A dict, command args. empty dict {} if there is no command args
        :Usage:
            ::

                driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': requestId})
        :Returns:
            A dict, empty dict {} if there is no result to return.
            For example to getResponseBody:
            {'base64Encoded': False, 'body': 'response body string'}
        """
        return self.execute("executeCdpCommand", {"cmd": cmd, "params": cmd_args})["value"]

    def get_sinks(self) -> list:
        """:Returns: A list of sinks available for Cast."""
        return self.execute("getSinks")["value"]

    def get_issue_message(self):
        """:Returns: An error message when there is any issue in a Cast
        session."""
        return self.execute("getIssueMessage")["value"]

    def set_sink_to_use(self, sink_name: str) -> dict:
        """Sets a specific sink, using its name, as a Cast session receiver
        target.

        :Args:
         - sink_name: Name of the sink to use as the target.
        """
        return self.execute("setSinkToUse", {"sinkName": sink_name})

    def start_desktop_mirroring(self, sink_name: str) -> dict:
        """Starts a desktop mirroring session on a specific receiver target.

        :Args:
         - sink_name: Name of the sink to use as the target.
        """
        return self.execute("startDesktopMirroring", {"sinkName": sink_name})

    def start_tab_mirroring(self, sink_name: str) -> dict:
        """Starts a tab mirroring session on a specific receiver target.

        :Args:
         - sink_name: Name of the sink to use as the target.
        """
        return self.execute("startTabMirroring", {"sinkName": sink_name})

    def stop_casting(self, sink_name: str) -> dict:
        """Stops the existing Cast session on a specific receiver target.

        :Args:
         - sink_name: Name of the sink to stop the Cast session.
        """
        return self.execute("stopCasting", {"sinkName": sink_name})

    def quit(self) -> None:
        """Closes the browser and shuts down the ChromiumDriver executable that
        is started when starting the ChromiumDriver."""
        try:
            super().quit()
        except Exception:
            # We don't care about the message because something probably has gone wrong
            pass
        finally:
            self.service.stop()
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


class WebDriver(ChromiumDriver):
    """Controls the ChromeDriver and allows you to drive the browser."""

    def __init__(
        self,
        options: Options = None,
        service: Service = None,
        keep_alive: bool = True,
    ) -> None:
        """Creates a new instance of the chrome driver. Starts the service and
        then creates new instance of chrome driver.

        :Args:
         - options - this takes an instance of ChromeOptions
         - service - Service object for handling the browser driver if you need to pass extra details
         - keep_alive - Whether to configure ChromeRemoteConnection to use HTTP keep-alive.
        """
        service = service if service else Service()
        options = options if options else Options()

        super().__init__(
            DesiredCapabilities.CHROME["browserName"],
            "goog",
            options,
            service,
            keep_alive,
        )
        
import logging
from pathlib import Path

from selenium.common.exceptions import NoSuchDriverException
from selenium.webdriver.common.options import BaseOptions
from selenium.webdriver.common.selenium_manager import SeleniumManager
from selenium.webdriver.common.service import Service

logger = logging.getLogger(__name__)


class _DriverFinder:
    """Utility to find if a given file is present and executable.

    This implementation is still in beta, and may change.
    """

    @staticmethod
    def get_path(service: Service, options: BaseOptions) -> str:
        path = service.path
        # path = None
        try:
            path = SeleniumManager().driver_location(options) if path is None else path
            print(path)
        except Exception as err:
            msg = f"Unable to obtain driver for {options.capabilities['browserName']} using Selenium Manager."
            raise NoSuchDriverException(msg) from err

        if path is None or not Path(path).is_file():
            raise NoSuchDriverException(f"Unable to locate or obtain driver for {options.capabilities['browserName']}")

        return path

NameError: name 'Options' is not defined

In [7]:
q_id = 625769474

In [6]:

# from selenium.webdriver import FirefoxService, FirefoxOptions
firefoxOptions = webdriver.FirefoxOptions()
firefoxOptions.binary_location = drivers["firefox"]
service = webdriver.FirefoxService(executable_path=drivers["firefox"])
driver = webdriver.Firefox(
    service=service,
    options=firefoxOptions
)

AttributeError: module 'selenium.webdriver' has no attribute 'FirefoxOptions'

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element(By.NAME, "q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
driver.close()

WebDriverException: Message: The browser appears to have exited before we could connect. If you specified a log_file in the FirefoxBinary constructor, check it for details.


In [21]:
print(Path(drivers["edge"]).is_file())

False


In [8]:
# service = webdriver.EdgeService(executable_path=drivers["edge"])
# edgeOptions = webdriver.EdgeOptions()
# edgeOptions.binary_location = drivers["edge"]
# edgeOptions.add_argument("--no-sandbox")
try:
    driver = webdriver.Edge(
        # service=service,
        # options=edgeOptions
    )
except Exception as e:
    print(e)
    raise e

Message: 'MicrosoftWebDriver.exe' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687 



WebDriverException: Message: 'MicrosoftWebDriver.exe' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687 


In [11]:
service = webdriver.ChromeService(executable_path=drivers["chrome"])
# options = Options()
chromeOptions = webdriver.ChromeOptions() 
chromeOptions.binary_location = drivers["chrome"]
# chromeOptions.add_argument("--no-sandbox")
# chromeOptions.add_argument("--remote-debugging-port=9222")
chromeOptions.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2}) 
chromeOptions.add_argument("--no-sandbox") 
# chromeOptions.add_argument("--disable-setuid-sandbox") 
# chromeOptions.add_argument("--disable-dev-shm-using") 
chromeOptions.add_argument("--disable-extensions") 
chromeOptions.add_argument("--disable-gpu") 
chromeOptions.add_argument("start-maximized") 
chromeOptions.add_argument("disable-infobars") 
chromeOptions.add_argument("--headless") 
# chromeOptions.add_argument(r"user-data-dir=.\cookies\\" + login) 
try: 
    driver = webdriver.Chrome(
        drivers["chrome"]
        #executable_path=drivers["chrome"]
        # service=service,
        # options =chromeOptions
    )
except Exception as e:
    print(e)
    raise e
# https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location/


AttributeError: module 'selenium.webdriver' has no attribute 'ChromeService'

In [12]:
# service = webdriver.ChromeService(executable_path=drivers["chrome"])
# options = Options()
# chromeOptions = webdriver.ChromeOptions() 
# chromeOptions.binary_location = drivers["chrome"]
try: 
    driver = webdriver.Chrome(
        drivers["chrome"]
        #executable_path=drivers["chrome"]
        # service=service,
        # options =chromeOptions
    )
except Exception as e:
    print(e)
    raise e
# https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location/


In [16]:
driver.get(f"https://www.zhihu.com/question/{q_id}")
print("get url")


In [ ]:
"newSession"

In [33]:
def _create_caps(caps):
    """Makes a W3C alwaysMatch capabilities object.

    Filters out capability names that are not in the W3C spec. Spec-compliant
    drivers will reject requests containing unknown capability names.

    Moves the Firefox profile, if present, from the old location to the new Firefox
    options object.

    :Args:
     - caps - A dictionary of capabilities requested by the caller.
    """
    caps = copy.deepcopy(caps)
    always_match = {}
    for k, v in caps.items():
        always_match[k] = v
    return {"capabilities": {"firstMatch": [{}], "alwaysMatch": always_match}}

cap = chromeOptions.to_capabilities()
print(cap)

{'browserName': 'chrome', 'pageLoadStrategy': 'normal', 'goog:chromeOptions': {'prefs': {'profile.managed_default_content_settings.images': 2}, 'extensions': [], 'binary': '/media/ywatcher/ExtDisk1/Files/Packages/web_archiv/driver/chromedriver-linux64/chromedriver', 'args': ['--no-sandbox', '--disable-extensions', '--disable-gpu', 'start-maximized', 'disable-infobars', '--headless']}}


In [17]:
import sqlite3
from sqlite3 import Error

In [26]:
if True:
    def create_connection(db_file):
        """ create a database connection to a SQLite database """
        conn = None
        try:
            conn = sqlite3.connect(db_file)
            print(sqlite3.version)
        except Error as e:
            print(e)
        finally:
            if conn:
                conn.close()
else:
    def create_connection():
        """ create a database connection to a database that resides
            in the memory
        """
        conn = None;
        try:
            conn = sqlite3.connect(':memory:')
            print(sqlite3.version)
        except Error as e:
            print(e)
        finally:
            if conn:
                conn.close()

In [27]:
create_connection(db_test)

2.6.0


In [28]:
db_file=db_test
conn = None
try:
    conn = sqlite3.connect(db_file)
    print(sqlite3.version)
except Error as e:
    print(e)

2.6.0


In [27]:
type(conn)

sqlite3.Connection

In [36]:
init_database(conn)

0

In [24]:
# answer_table:
title_4_answer_table = "Answers"
title_4_question_table = "Questions"
title_4_comment_table = "Comments"
query_create_answer_table = f"""
CREATE TABLE IF NOT EXISTS {title_4_answer_table} (
    id integer PRIMARY KEY,
    authorId integer,
    questionId integer NOT NULL,
    content text
    )
"""
query_create_answer_vcs_table = f"""
CREATE TABLE IF NOT EXISTS {title_4_answer_table}VCS (
    commitId text PRIMARY KEY,
    answerId interger NOT NULL,
    dateFetched text,
    dateAmended text,
    contentDiff text
    )
"""
query_create_question_table = f"""
CREATE TABLE IF NOT EXISTS {title_4_question_table} (
    id integer PRIMARY KEY,
    authorId integer
    )
"""
query_create_question_vcs_table = f"""
CREATE TABLE IF NOT EXISTS {title_4_question_table}VCS (
    commitId text PRIMARY KEY,
    questionId interger NOT NULL,
    dateFetched text,
    dateAmended text,
    titleDiff text,
    contentDiff text
    )
"""
# create a table named GitHead
# to maintain the current head
# of git repository, sha of 
# which is stored;
# This table aims to prevent
# unexpected effects caused by
# manual git operations
query_create_head_table = f"""
CREATE TABLE GitHead (
    id INTEGER PRIMARY KEY CHECK (id = 0),
    sha text
);
CREATE TRIGGER git_head_no_insert
BEFORE INSERT ON GitHead
WHEN (SELECT COUNT(*) FROM GitHead) >= 1   -- limit here
BEGIN
    SELECT RAISE(FAIL, 'only one row!');
END;
"""


def init_database_file(
    db_file:str
):
    pass

def init_database_tables(
    conn: sqlite3.Connection
) -> int:
    # TODO: create database file,
    # assert that file does not exist
    # if forced create: overwrite=True
    try: 
        c = conn.cursor()
        c.execute(query_create_question_table)
        c.execute(query_create_answer_vcs_table)
        c.execute(query_create_answer_table)
        c.execute(query_create_question_vcs_table)
        c.executescript(query_create_head_table)
        return 0
    except Error as e:
        print(e)
        return -1
    

In [29]:
init_database(conn)

0

https://stackoverflow.com/questions/33104101/ensure-sqlite-table-only-has-one-row

In [48]:
git_repo = os.path.join(
    root, "git_repo"
)
os.path.exists(git_repo)
git_repo = "/media/ywatcher/ExtDisk1/Files/web_archiv_/git_repo"
# you should exec git init in this repo

In [45]:
from git import *

In [47]:
repo = Repo(git_repo) 

In [49]:
repo

<git.repo.base.Repo '/media/ywatcher/ExtDisk1/Files/web_archiv_/git_repo/.git'>

In [50]:
repo.active_branch

<git.Head "refs/heads/master">

<bound method Repo.archive of <git.repo.base.Repo '/media/ywatcher/ExtDisk1/Files/web_archiv_/git_repo/.git'>>

In [ ]:
def update_answer(a:"Answer"):
    # if not in db
    # create a record in db
    # create a file in git repo
    # else
    # get current file 
    # check unstaged change
    # make sure no change unstaged
    # overwrite file with current content
    # overwrite comment list with current
    # overwrite title if any
    # commit with fetched date
    # get a new node for the file
    # save the node to vcs table:
    # fetched date
    # node id 
    # and 
    pass

how to merge db:
extract content from git repo
1. 
create a new field to your vcs table
indicate the git repo it belongs to;
the identifier should be commit hash<br>
2. 
merge vcs tables<br>
3.
sort vcs table by date<br>
4.
for each node, get full content from git repo
and commit to new git repo<br>

---

before merge, you can indicate the tag of each git repo <br>
todo.


